In [49]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.express as px

In [34]:
# 데이터를 불러옵니다.
donation = pd.read_excel('donations_new.xlsx')
general = pd.read_excel('general_new.xlsx')

In [35]:
#원하는 컬럼만 추출
general = general[['chid','title','created_date','funds_raised', 'funds_target', 'total_donations']]
general.head()

,chid,title,created_date,funds_raised,funds_target,total_donations
0,1,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,1705.0
1,2,1 percent chance,2020-06-21,299042,2000000.0,602.0
2,3,MELS needs #100ComputersIn100Days,2020-06-19,13845,30000.0,125.0
3,4,100% to Victims of Taal Volcano in Philippines,2020-01-16,25,5000.0,1.0
4,5,$100k for Center for Family Life,2020-11-01,13950,100000.0,17.0


In [36]:
# 원하는 컬럼만 추출
donation = donation[['chid','amount','created_at']]

In [37]:
# created_at 시간스탬프 삭제 및 컬럼명 조정
donation['created_at'] = donation['created_at'].dt.date
donation.rename(columns = {'created_at' : 'donated_at'}, inplace = True)

In [66]:
donation['donated_at'] = pd.to_datetime(donation['donated_at'])

In [73]:
donation.head()

,chid,amount,donated_at
0,1,480,2020-10-14
1,1,10,2020-09-14
2,1,50,2020-09-09
3,1,100,2020-09-09
4,1,100,2020-09-07


In [84]:
# general과 donation 테이블 inner join
# 펀딩 성공여부를 알 수 있는 불리언 타입의 'is_successful' 컬럼 추가
df_1 = pd.merge(general, donation, how = 'inner', on = 'chid')
df = df_1.assign(is_successful = df_1['funds_raised'] >= df_1['funds_target'],
                date_diff = df_1['donated_at'] - df_1['created_date'])
df.dtypes

chid                         int64
title                       object
created_date        datetime64[ns]
funds_raised                 int64
funds_target               float64
total_donations            float64
amount                       int64
donated_at          datetime64[ns]
is_successful                 bool
date_diff          timedelta64[ns]
dtype: object

In [85]:
# timedelta인 date_diff를 숫자형(float)으로 변환 (그래프 x축에 두기 위해)
df['date_diff'] = df['date_diff'].apply(lambda x:x.days)

In [76]:
df.dtypes

chid                        int64
title                      object
created_date       datetime64[ns]
funds_raised                int64
funds_target              float64
total_donations           float64
amount                      int64
donated_at         datetime64[ns]
is_successful                bool
date_diff                 float64
dtype: object

In [86]:
# 랜덤으로 chid 693번 펀딩의 정보 확인
df.loc[df['chid'] == 693,:]

,chid,title,created_date,funds_raised,funds_target,total_donations,amount,donated_at,is_successful,date_diff
403212,693,Black Leadership Initiative,2020-06-26,15847,15000.0,209.0,100,2020-07-21,True,25.0
403213,693,Black Leadership Initiative,2020-06-26,15847,15000.0,209.0,25,2020-07-13,True,17.0
403214,693,Black Leadership Initiative,2020-06-26,15847,15000.0,209.0,20,2020-07-10,True,14.0
403215,693,Black Leadership Initiative,2020-06-26,15847,15000.0,209.0,500,2020-07-10,True,14.0
403216,693,Black Leadership Initiative,2020-06-26,15847,15000.0,209.0,25,2020-07-09,True,13.0
...,...,...,...,...,...,...,...,...,...,...
403416,693,Black Leadership Initiative,2020-06-26,15847,15000.0,209.0,100,2020-06-26,True,0.0
403417,693,Black Leadership Initiative,2020-06-26,15847,15000.0,209.0,150,2020-06-26,True,0.0
403418,693,Black Leadership Initiative,2020-06-26,15847,15000.0,209.0,100,2020-06-26,True,0.0
403419,693,Black Leadership Initiative,2020-06-26,15847,15000.0,209.0,100,2020-06-26,True,0.0


In [79]:
# 랜덤 chid의 펀딩의 날짜별 기부 횟수(donation_no)와 금액(amount), 펀딩시작일로부터의 일자 차이 확인
df_0233 = df.query('chid == 233').groupby('donated_at').agg({'title':'count','amount':'sum','date_diff':'mean'}).rename(columns = {'title' : 'donation_no'}).sort_values(by = 'donated_at', ascending = True).reset_index()
df_0292 = df.query('chid == 292').groupby('donated_at').agg({'title':'count','amount':'sum','date_diff':'mean'}).rename(columns = {'title' : 'donation_no'}).sort_values(by = 'donated_at', ascending = True).reset_index()
df_0478 = df.query('chid == 478').groupby('donated_at').agg({'title':'count','amount':'sum','date_diff':'mean'}).rename(columns = {'title' : 'donation_no'}).sort_values(by = 'donated_at', ascending = True).reset_index()
df_0693 = df.query('chid == 693').groupby('donated_at').agg({'title':'count','amount':'sum','date_diff':'mean'}).rename(columns = {'title' : 'donation_no'}).sort_values(by = 'donated_at', ascending = True).reset_index()

In [87]:
# chid 233 케이스 데이터 확인
df_0233

,donated_at,donation_no,amount,date_diff
0,2020-10-15,355,45571,0.0
1,2020-10-16,139,17830,1.0
2,2020-10-17,29,4450,2.0
3,2020-10-18,21,1809,3.0
4,2020-10-19,20,1390,4.0
5,2020-10-20,12,1395,5.0
6,2020-10-21,8,570,6.0
7,2020-10-22,2,55,7.0
8,2020-10-23,8,720,8.0
9,2020-10-24,2,100,9.0


In [78]:
# chid 233 케이스 그래프 확인
fig = px.bar(df_0233, x='date_diff', y='donation_no', color='amount', color_continuous_scale='Greens', height=400)
fig.show()

In [80]:
# chid 292 케이스 그래프 확인
fig = px.bar(df_0292, x='date_diff', y='donation_no', color='amount', color_continuous_scale='Greens', height=400)
fig.show()

In [81]:
# chid 478 케이스 그래프 확인
fig = px.bar(df_0478, x='date_diff', y='donation_no', color='amount', color_continuous_scale='Greens', height=400)
fig.show()

In [82]:
# chid 693 케이스 그래프 확인
fig = px.bar(df_0693, x='date_diff', y='donation_no', color='amount', color_continuous_scale='Greens', height=400)
fig.show()

In [88]:
df

,chid,title,created_date,funds_raised,funds_target,total_donations,amount,donated_at,is_successful,date_diff
0,1,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,1705.0,480,2020-10-14,False,133.0
1,1,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,1705.0,10,2020-09-14,False,103.0
2,1,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,1705.0,50,2020-09-09,False,98.0
3,1,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,1705.0,100,2020-09-09,False,98.0
4,1,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,1705.0,100,2020-09-07,False,96.0
...,...,...,...,...,...,...,...,...,...,...
1048538,1848,Honoring James Bostic,2020-08-25,40472,40000.0,450.0,10,2020-08-27,True,2.0
1048539,1848,Honoring James Bostic,2020-08-25,40472,40000.0,450.0,25,2020-08-27,True,2.0
1048540,1848,Honoring James Bostic,2020-08-25,40472,40000.0,450.0,50,2020-08-27,True,2.0
1048541,1848,Honoring James Bostic,2020-08-25,40472,40000.0,450.0,25,2020-08-27,True,2.0


In [89]:
# KMeans clustering으로 모금형태 구분 시도 예정
from sklearn.cluster import KMeans

'''
X = df[['kills', 'assists', 'damage']]
kmeans = KMeans(n_clusters=8, random_state=0).fit(X)

fig = px.scatter(
    개인유효공격, x = 'damage', y='kills', hover_data=['player_id'], color=kmeans.labels_)
fig.show()
'''